In [36]:
import rasterio, numpy as np
from ipywidgets import FloatSlider, IntSlider, interact, Layout
from rasterio.warp import Resampling
import numpy as np
import matplotlib.pyplot as plt
import os

In [37]:
# 1. Paths to your rasters (ML map + distance layers + binary mask)

ML_path = r"C:\Users\TyHow\MinersAI Dropbox\Tyler Howe\Sibelco_Stuff\linear_combination_layers\averaged_probability_map_smoothed_thresholded_95th_percentile.tif"
roads_path = r"C:\Users\TyHow\MinersAI Dropbox\Tyler Howe\Sibelco_Stuff\linear_combination_layers\infrastructure\roads_vector.tif"
power_path = r"C:\Users\TyHow\MinersAI Dropbox\Tyler Howe\Sibelco_Stuff\linear_combination_layers\infrastructure\powerlines_vector.tif"
rail_path = r"C:\Users\TyHow\MinersAI Dropbox\Tyler Howe\Sibelco_Stuff\linear_combination_layers\infrastructure\railways_vector.tif"
developed_mask_path = r"C:\Users\TyHow\MinersAI Dropbox\Tyler Howe\Sibelco_Stuff\linear_combination_layers\infrastructure\town_mask_continuous.tif"

paths = [ML_path, roads_path, power_path, rail_path, developed_mask_path]

In [40]:
# open first file to define target grid
with rasterio.open(paths[0]) as ref:
    ref_meta = ref.meta.copy()
    target_height = ref.height
    target_width = ref.width
    target_transform = ref.transform
    target_crs = ref.crs
    H, W = ref.height, ref.width

arrays = []
for p in paths:
    with rasterio.open(p) as src:
        resamp = Resampling.nearest if p == developed_mask_path else Resampling.bilinear
        arr = src.read(1, out_shape=(H, W), resampling=resamp).astype(float)
        arrays.append(arr)

# separate out town mask index
infra_maps = arrays[1:]
infra_names = [os.path.splitext(os.path.basename(p))[0] for p in paths[1:]]
mins = [float(np.nanmin(m)) for m in infra_maps]
maxs = [float(np.nanmax(m)) for m in infra_maps]

xmin = ref_meta["transform"][2]
ymax = ref_meta["transform"][5]
xmax = xmin + ref_meta["transform"][0] * target_width
ymin = ymax + ref_meta["transform"][4] * target_height
extent = (xmin, xmax, ymin, ymax)

In [ ]:
# separate ML base and infra layers
ml_map = arrays[0]
infra_maps = arrays[1:]
infra_names = [os.path.splitext(os.path.basename(p))[0] for p in paths[1:]]

# compute slider ranges from the raw infra maps
infra_mins = [float(np.nanmin(m)) for m in infra_maps]
infra_maxs = [float(np.nanmax(m)) for m in infra_maps]


def show_map(**thresholds):
    # start with the ML map
    filtered = ml_map.copy()
    # for each infra layer, zero out ML where infra > threshold
    for name, layer in zip(infra_names, infra_maps):
        t = thresholds[name]
        filtered[layer > t] = 0
    # plot
    plt.figure(figsize=(12, 10))
    plt.imshow(filtered, extent=extent, origin="upper", cmap="viridis")
    plt.colorbar(label="Filtered ML probability")
    plt.axis("off")
    plt.title("ML output within user‐defined cutoffs")


# build one slider per infra layer, using its raw min/max
DESC_W, WIDGET_W = "180px", "450px"
# build sliders
sliders = {}
for name, mn, mx in zip(infra_names, mins, maxs):
    if name == os.path.splitext(os.path.basename(developed_mask_path))[0]:
        # integer counts
        sliders[name] = IntSlider(
            value=int(mx),
            min=int(mn),
            max=int(mx),
            step=1,
            description=name,
            style={"description_width": "180px"},
            layout=Layout(width="450px"),
        )
    else:
        sliders[name] = FloatSlider(
            value=mx,
            min=mn,
            max=mx,
            step=(mx - mn) / 100 or 1,
            description=name,
            style={"description_width": "180px"},
            layout=Layout(width="450px"),
        )


def show_map(**thresholds):
    filtered = ml_map.copy()
    for name, layer in zip(infra_names, infra_maps):
        t = thresholds[name]
        filtered[layer > t] = 0
    plt.figure(figsize=(12, 10))
    plt.imshow(filtered, extent=extent, origin="upper", cmap="viridis")
    plt.axis("off")


interact(show_map, **sliders)

interactive(children=(FloatSlider(value=92.90148162841797, description='roads_vector', layout=Layout(width='45…

<function __main__.show_map(**thresholds)>

In [42]:
# 1) Re-load your ML metadata so we preserve CRS, transform, etc.
with rasterio.open(ML_path) as src:
    out_meta = src.meta.copy()
# we’re writing one band of floats
out_meta.update(dtype=rasterio.float32, count=1)

# 2) Recompute the filtered map using your sliders’ .value
filtered = ml_map.copy()
for name, layer in zip(infra_names, infra_maps):
    thresh = sliders[name].value
    # zero out ML where infra > slider threshold
    filtered[layer > thresh] = 0

# 3) Write it out
out_path = r"C:\Users\TyHow\MinersAI Dropbox\Tyler Howe\Sibelco_Stuff\linear_combination_layers\combination.tif"
with rasterio.open(out_path, "w", **out_meta) as dst:
    dst.write(filtered.astype("float32"), 1)

print(f"Saved filtered map to {out_path}")

Saved filtered map to C:\Users\TyHow\MinersAI Dropbox\Tyler Howe\Sibelco_Stuff\linear_combination_layers\combination.tif
